# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 5 2022 10:14AM,243539,19,ADV80004430,"AdvaGen Pharma, Ltd",Released
1,Aug 5 2022 10:14AM,243539,19,ADV80004455,"AdvaGen Pharma, Ltd",Executing
2,Aug 5 2022 10:01AM,243538,16,SSF-8364694,Sartorius Stedim Filters Inc.,Released
3,Aug 5 2022 9:58AM,243537,12,U0982,"Uniderm USA, Inc",Released
4,Aug 5 2022 9:58AM,243537,12,U0983,"Uniderm USA, Inc",Released
5,Aug 5 2022 9:58AM,243537,12,U0984,"Uniderm USA, Inc",Released
6,Aug 5 2022 9:58AM,243537,12,U0985,"Uniderm USA, Inc",Released
7,Aug 5 2022 9:48AM,243536,16,SSF-8364680,Sartorius Stedim Filters Inc.,Released
8,Aug 5 2022 9:32AM,243535,12,HH6518,Hush Hush,Released
9,Aug 5 2022 9:32AM,243534,12,HH-36252,Hush Hush,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,243536,Released,1
31,243537,Released,4
32,243538,Released,1
33,243539,Executing,1
34,243539,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243535,NaN,1.0
243536,NaN,1.0
243537,NaN,4.0
243538,NaN,1.0
243539,1.0,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243451,1.0,0.0
243455,0.0,1.0
243461,19.0,0.0
243464,7.0,6.0
243478,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243451,1,0
243455,0,1
243461,19,0
243464,7,6
243478,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243451,1,0
1,243455,0,1
2,243461,19,0
3,243464,7,6
4,243478,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243451,1,
1,243455,,1
2,243461,19,
3,243464,7,6
4,243478,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 5 2022 10:14AM,243539,19,"AdvaGen Pharma, Ltd"
2,Aug 5 2022 10:01AM,243538,16,Sartorius Stedim Filters Inc.
3,Aug 5 2022 9:58AM,243537,12,"Uniderm USA, Inc"
7,Aug 5 2022 9:48AM,243536,16,Sartorius Stedim Filters Inc.
8,Aug 5 2022 9:32AM,243535,12,Hush Hush
9,Aug 5 2022 9:32AM,243534,12,Hush Hush
18,Aug 5 2022 9:28AM,243533,16,Sartorius Stedim Filters Inc.
19,Aug 5 2022 9:21AM,243532,16,Sartorius Stedim Filters Inc.
20,Aug 5 2022 9:19AM,243531,10,Emerginnova
21,Aug 5 2022 9:18AM,243530,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 5 2022 10:14AM,243539,19,"AdvaGen Pharma, Ltd",1,1
1,Aug 5 2022 10:01AM,243538,16,Sartorius Stedim Filters Inc.,,1
2,Aug 5 2022 9:58AM,243537,12,"Uniderm USA, Inc",,4
3,Aug 5 2022 9:48AM,243536,16,Sartorius Stedim Filters Inc.,,1
4,Aug 5 2022 9:32AM,243535,12,Hush Hush,,1
5,Aug 5 2022 9:32AM,243534,12,Hush Hush,6,3
6,Aug 5 2022 9:28AM,243533,16,Sartorius Stedim Filters Inc.,,1
7,Aug 5 2022 9:21AM,243532,16,Sartorius Stedim Filters Inc.,,1
8,Aug 5 2022 9:19AM,243531,10,Emerginnova,,1
9,Aug 5 2022 9:18AM,243530,10,Eywa Pharma Inc.,1,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 10:14AM,243539,19,"AdvaGen Pharma, Ltd",1,1
1,Aug 5 2022 10:01AM,243538,16,Sartorius Stedim Filters Inc.,1,
2,Aug 5 2022 9:58AM,243537,12,"Uniderm USA, Inc",4,
3,Aug 5 2022 9:48AM,243536,16,Sartorius Stedim Filters Inc.,1,
4,Aug 5 2022 9:32AM,243535,12,Hush Hush,1,
5,Aug 5 2022 9:32AM,243534,12,Hush Hush,3,6
6,Aug 5 2022 9:28AM,243533,16,Sartorius Stedim Filters Inc.,1,
7,Aug 5 2022 9:21AM,243532,16,Sartorius Stedim Filters Inc.,1,
8,Aug 5 2022 9:19AM,243531,10,Emerginnova,1,
9,Aug 5 2022 9:18AM,243530,10,Eywa Pharma Inc.,7,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 10:14AM,243539,19,"AdvaGen Pharma, Ltd",1,1
1,Aug 5 2022 10:01AM,243538,16,Sartorius Stedim Filters Inc.,1,
2,Aug 5 2022 9:58AM,243537,12,"Uniderm USA, Inc",4,
3,Aug 5 2022 9:48AM,243536,16,Sartorius Stedim Filters Inc.,1,
4,Aug 5 2022 9:32AM,243535,12,Hush Hush,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 10:14AM,243539,19,"AdvaGen Pharma, Ltd",1.0,1.0
1,Aug 5 2022 10:01AM,243538,16,Sartorius Stedim Filters Inc.,1.0,NaN
2,Aug 5 2022 9:58AM,243537,12,"Uniderm USA, Inc",4.0,NaN
3,Aug 5 2022 9:48AM,243536,16,Sartorius Stedim Filters Inc.,1.0,NaN
4,Aug 5 2022 9:32AM,243535,12,Hush Hush,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 10:14AM,243539,19,"AdvaGen Pharma, Ltd",1.0,1.0
1,Aug 5 2022 10:01AM,243538,16,Sartorius Stedim Filters Inc.,1.0,0.0
2,Aug 5 2022 9:58AM,243537,12,"Uniderm USA, Inc",4.0,0.0
3,Aug 5 2022 9:48AM,243536,16,Sartorius Stedim Filters Inc.,1.0,0.0
4,Aug 5 2022 9:32AM,243535,12,Hush Hush,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2191546,78.0,33.0
12,974103,8.0,7.0
15,243509,1.0,0.0
16,1461164,6.0,0.0
19,486990,1.0,2.0
20,1217480,4.0,26.0
21,243483,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2191546,78.0,33.0
1,12,974103,8.0,7.0
2,15,243509,1.0,0.0
3,16,1461164,6.0,0.0
4,19,486990,1.0,2.0
5,20,1217480,4.0,26.0
6,21,243483,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,78.0,33.0
1,12,8.0,7.0
2,15,1.0,0.0
3,16,6.0,0.0
4,19,1.0,2.0
5,20,4.0,26.0
6,21,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,78.0
1,12,Released,8.0
2,15,Released,1.0
3,16,Released,6.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,33.0,7.0,0.0,0.0,2.0,26.0,1.0
Released,78.0,8.0,1.0,6.0,1.0,4.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,33.0,7.0,0.0,0.0,2.0,26.0,1.0
1,Released,78.0,8.0,1.0,6.0,1.0,4.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,33.0,7.0,0.0,0.0,2.0,26.0,1.0
1,Released,78.0,8.0,1.0,6.0,1.0,4.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()